# Intro

In this notebook, we aim to merge multiple datasets into one df in order to do further analysis.

We aim to get the 'text' column of what we will be labelling as `text_df` , append a date via the `date_df`.

# Imports

In [1]:
import pandas as pd

# Load Datasets

**Including Text**

https://www.kaggle.com/datasets/michaelrussell4/10000-books-and-their-genres-standardized

In [2]:
text_df = pd.read_csv('datasets/books_and_genres.csv')

**Including Publication Date**

https://www.kaggle.com/datasets/saurabhbagchi/books-dataset

In [3]:
date_df = pd.read_csv('datasets/books.csv', encoding='ISO-8859-1', on_bad_lines='skip', sep=';', dtype={'Year-Of-Publication': 'str'})

# Basic Overview

In [4]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10635 entries, 0 to 10634
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10635 non-null  int64 
 1   title       10635 non-null  object
 2   text        10373 non-null  object
 3   genres      10635 non-null  object
dtypes: int64(1), object(3)
memory usage: 332.5+ KB


In [5]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
 5   Image-URL-S          271360 non-null  object
 6   Image-URL-M          271360 non-null  object
 7   Image-URL-L          271357 non-null  object
dtypes: object(8)
memory usage: 16.6+ MB


# Checking for missing data, dropping missing text

In [6]:
print(text_df.isnull().sum(), '\n\n', date_df.isnull().sum())

Unnamed: 0      0
title           0
text          262
genres          0
dtype: int64 

 ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


From the above, we see 262 rows are missing `text` , and no `Year of Publication` is missing.

We will delete the rows where text is missing.

In [7]:
text_df = text_df.dropna(subset=['text'])
date_df = date_df.dropna(subset=['Book-Author'])

# Dropping Unnecessary Columns

In [8]:
text_df = text_df.drop(columns=['Unnamed: 0'])

In [9]:
date_df = date_df.drop(columns=['ISBN', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'])